# User input; only parameter

In [21]:
user_input="C:/Users/A02315404/Downloads/Fischer_towers.gpx"


# Importing modules, naming data folders

In [35]:
#modules (environ = raster_proj)
import arcpy
import os
#import classtools
arcpy.CheckOutExtension('spatial')
from arcpy import env
from arcpy.sa import *
import rasterio
import csv
import numpy as np
import gdal

arcpy.overwriteoutput = True

filename = os.path.split(user_input)[1]
filename = filename.replace(".gpx", "")

template = r'C:\Users\A02315404\Documents\python\frolic_analysis\data\ecoregionsL4.shp'

#folders
conv_data_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_converted'
data_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data'
data_orig_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_orig'
int_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\intersections'
in_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_gpx_files'
out_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis\data_converted'
frolic_folder = r'C:\Users\A02315404\Documents\python\frolic_analysis'

arcpy.env.workspace = data_folder

#shape/image data
rock = os.path.join(data_orig_folder, "rock.tif")
nlcd = os.path.join(data_orig_folder, "nlcd.tif")
nlcd_key = os.path.join(data_orig_folder, "nlcd_key.csv")
rock_key = os.path.join(data_orig_folder, "rock_key.csv")
eco3 = os.path.join(data_orig_folder, "ecoregionsL3.shp")
eco4 = os.path.join(data_orig_folder, "ecoregionsL4.shp")


#trail (from user input)
trail =  os.path.join(conv_data_folder, filename) + ".shp"
output_name_shp = os.path.join(out_folder, filename)+ ".shp"
output_name_tif = os.path.join(out_folder, filename)+ ".tif"
out_name_nlcd=os.path.join(int_folder, filename) + "_nlcd.tif"
out_name_rock=os.path.join(int_folder, filename) + "_rock.tif"
temp_fn = os.path.join(frolic_folder, filename) + '_test_maps.pdf'
pdf_fn = os.path.join(frolic_folder, filename) + '_maps.pdf'


# Converting user_input (.gpx) to .tif and  .shp files

In [23]:
#print(filename,"\n", output_name_shp, "\n", output_name_tif)
arcpy.GPXtoFeatures_conversion(user_input, output_name_shp)
arcpy.conversion.PointToRaster(in_features=output_name_shp, 
                               value_field='FID', 
                               out_rasterdataset=output_name_tif)


Fischer_towers 
 C:\Users\A02315404\Documents\python\frolic_analysis\data_converted\Fischer_towers.shp 
 C:\Users\A02315404\Documents\python\frolic_analysis\data_converted\Fischer_towers.tif


<Result 'C:\\Users\\A02315404\\Documents\\python\\frolic_analysis\\data_converted\\Fischer_towers.tif'>

# Raster Stats 

In [30]:
#make dictionaries from csv files
nlcd_dict = {}
with open(nlcd_key, mode='r') as inp:
    reader = csv.reader(inp)
    nlcd_dict = {rows[0]:rows[1] for rows in reader}

rock_dict = {}
with open(rock_key, mode='r') as inp:
    reader = csv.reader(inp)
    rock_dict = {rows[0]:rows[1] for rows in reader}

#define output intersections    
data_list = [trail,rock,nlcd]
out_name=os.path.join(int_folder, filename) + ".tif"

#use ExtractByMask to make intersections
outExtractByMask=ExtractByMask(in_raster=nlcd, in_mask_data=trail)
outExtractByMask.save(out_name_nlcd)
outExtractByMask=ExtractByMask(in_raster=rock, in_mask_data=trail)
outExtractByMask.save(out_name_rock)

#get unique values from rasters
rock_np=gdal.Open(out_name_rock).GetRasterBand(1)
rock_min = str(round(Raster(out_name_rock).minimum))
rock_max = str(round(Raster(out_name_rock).maximum))
rock_avg = str(round(Raster(out_name_rock).mean))
rock_vals =np.unique(np.array([rock_min, rock_max, rock_avg]))
nlcd_np=gdal.Open(out_name_nlcd).GetRasterBand(1)
nlcd_min = str(round(Raster(out_name_nlcd).minimum))
nlcd_max = str(round(Raster(out_name_nlcd).maximum))
nlcd_avg = str(round(Raster(out_name_nlcd).mean))
nlcd_vals =np.unique(np.array([nlcd_min, nlcd_max, nlcd_avg]))

#print results
for r in rock_vals:
    print(" You hiked in the", rock_dict[str(r)], "zone of lithology!")
for n in nlcd_vals:
    print(" You hiked in the", nlcd_dict[str(n)], "landcover classification zone!")



 You hiked in the non-carbonate zone of lithology!
 You hiked in the Shrub landcover classification zone!


# Shp Stats

In [36]:
#make intersection
data_list = [trail,eco3,eco4]
out_name=os.path.join(int_folder, filename) + ".shp"
print(out_name)
arcpy.analysis.Intersect(in_features=data_list, 
                         out_feature_class=out_name, 
                         )
#funtion to get unique fieldnames
def unique_values(table , field):
    with arcpy.da.SearchCursor(table, [field]) as cursor:
        return sorted({row[0] for row in cursor})

eco4_vals = unique_values(out_name , 'us_l4name')
eco3_vals =unique_values(out_name , 'na_l3name')
#make dictionaries of results
eco4_dict = {}
eco3_dict = {}
count_all = int(arcpy.management.GetCount(out_name).getOutput(0))
count=0
for eco4_name in eco4_vals:
    subset_eco4 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view= out_name, 
        where_clause=f"us_l4name LIKE  '{eco4_vals[count]}' "
    )
    percent_eco4 = ((int(arcpy.management.GetCount(subset_eco4).getOutput(0)))/int(count_all)*100 )
    eco4_dict[eco4_vals[count]] = percent_eco4
    count=count+1
count=0
for eco3_name in eco3_vals:
    subset_eco3 = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view= out_name, 
        where_clause=f"na_l3name LIKE  '{eco3_vals[count]}' "
    )
    percent_eco3 = ((int(arcpy.management.GetCount(subset_eco3).getOutput(0)))/int(count_all)*100 )
    eco3_dict[eco3_vals[count]] = percent_eco3
    count=count+1
#print results
count=0
for eco in eco3_dict.items():
    if count==0:
        print(f"Your adventure took you through the {eco[0]} ( {round(eco[1])} % ) ") 
    if count!=0:
        print(f"and the {eco[0]} ( {round(eco[1])} % ) ") 
    count=count+1
count=0
for eco in eco4_dict.items():
    if count==0:
        print(f"specifically through the {(eco[0])} ( {round(eco[1])} % ) ")
    if count!=0:
        print(f" and the {(eco[0])} ( {round(eco[1])} % ) ") 
    count=count+1


C:\Users\A02315404\Documents\python\frolic_analysis\intersections\Fischer_towers.shp


NameError: name 'eco2_dict' is not defined

# Make maps

- will this only work if run from arcgis??

In [38]:
# Get the current project.
project = arcpy.mp.ArcGISProject("CURRENT")
[m.name for m in project.listMaps()]
# Get the currently active map.
mapp = project.activeMap

# Get the  layers.
eco4 = mapp.listLayers('eco4_crop')[0]
eco2 = mapp.listLayers('eco3_crop')[0]
rock = mapp.listLayers('rock_crop.tif')[0]
nlcd = mapp.listLayers('nlcd_crop.tif')[0]
lyr_list = [eco2,eco4,rock,nlcd]
print(lyr_list)

trail_lyr = mapp.listLayers(filename)[0]

lyr_names = ["Ecoregion Classificaiton III","Ecoregion Classificaiton IV","Type of Lithology" , "Landcover"]

OSError: CURRENT